# Task 1

In [ ]:
using Bloqade
using KrylovKit
using SparseArrays
using PythonCall

plt = pyimport("matplotlib.pyplot");

## Ground State Properties

In [ ]:
nsites = 9
atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)

In [ ]:
Ω = 2π * 4
Δ_step = 30
Δ = LinRange(-2π * 10, 2π * 10, Δ_step);

In [ ]:
density_g = zeros(Δ_step, nsites)

for ii in 1:Δ_step
    h_ii = rydberg_h(atoms; Δ = Δ[ii], Ω) # create the Rydberg Hamiltonian
    h_m = mat(h_ii) # convert the Hamiltonian into a matrix
    vals, vecs, info = KrylovKit.eigsolve(h_m, 1, :SR) # find the ground state eigenvalue and eigenvector
    g_state = ArrayReg(vecs[1]) # creates the initial state with all atoms in ``| 0 \rangle`` state

    for jj in 1:nsites
        density_g[ii, jj] = rydberg_density(g_state, jj) # measure the density of Rydberg excitations on each site
    end
end

In [ ]:
fig, ax = plt.subplots(figsize = (10, 4))
ax.bar(1:nsites, density_g[1, :])
ax.set_xticks(1:nsites)
ax.set_xlabel("Sites")
ax.set_ylabel("Rydberg density")
ax.set_title("Density Profile: 1D Chain, Δ = -2π * 10 MHz")
fig

In [ ]:
fig, ax = plt.subplots(figsize = (10, 4))
ax.bar(1:nsites, density_g[30, :])
ax.set_xticks(1:nsites)
ax.set_xlabel("Sites")
ax.set_ylabel("Rydberg density")
ax.set_title("Density Profile: 1D Chain, Δ = 2π * 10 MHz")
fig

In [ ]:
order_para = map(1:Δ_step) do ii
    return sum(density_g[ii, 1:2:nsites]) - sum(density_g[ii, 2:2:nsites])
end

fig, ax = plt.subplots(figsize = (10, 4))
ax.plot(Δ / 2π, order_para)
ax.set_xlabel("Δ/2π (MHz) ")
ax.set_ylabel("Order parameter")
fig

## Preparation of Ordered States in 1D

In [ ]:
total_time = 3.0;
Ω_max = 2π * 4;
Ω = piecewise_linear(clocks = [0.0, 0.1, 2.1, 2.2, total_time], values = [0.0, Ω_max, Ω_max, 0, 0]);

In [ ]:
U1 = -2π * 10;
U2 = 2π * 10;
Δ = piecewise_linear(clocks = [0.0, 0.6, 2.1, total_time], values = [U1, U1, U2, U2]);

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize = (12, 4))
Bloqade.plot!(ax1, Ω)
ax1.set_ylabel("Ω/2π (MHz)")
Bloqade.plot!(ax2, Δ)
ax2.set_ylabel("Δ/2π (MHz)")
fig

In [ ]:
nsites = 9
atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)

In [ ]:
h = rydberg_h(atoms; Δ, Ω)

In [ ]:
reg = zero_state(nsites);
prob = SchrodingerProblem(reg, total_time, h);
integrator = init(prob, Vern8());

In [ ]:
densities = []
σxs = []

for _ in TimeChoiceIterator(integrator, 0.0:1e-3:total_time)
    push!(densities, rydberg_density(reg))
    
    # Calculate expectation value of Xᵢ at each site
    σx = [expect(chain(nsites, put(i=>Op.X)), reg) for i in 1:nsites]
    push!(σxs, σx)
    
    
end
Dvals = hcat(densities...);
Xvals = hcat(σxs...);

In [ ]:
fig, ax = plt.subplots(figsize = (10, 4))
shw = ax.imshow(real(Dvals), interpolation = "nearest", aspect = "auto", extent = [0, total_time, 0.5, nsites + 0.5])
ax.set_xlabel("time (μs)")
ax.set_ylabel("site")
ax.set_xticks(0:0.2:total_time)
ax.set_yticks(1:nsites)
bar = fig.colorbar(shw)
fig

In [ ]:
fig, ax = plt.subplots(figsize = (10, 4))
shw = ax.imshow(real(Xvals), interpolation = "nearest", aspect = "auto", extent = [0, total_time, 0.5, nsites + 0.5])
ax.set_xlabel("time (μs)")
ax.set_ylabel("site")
ax.set_xticks(0:0.2:total_time)
ax.set_yticks(1:nsites)
bar = fig.colorbar(shw)
fig

In [ ]:
bitstring_hist(reg; nlargest = 20)

## Calculate Ground State Energy

In [ ]:
h_ryd = h |> attime(total_time)

In [ ]:
vals, vecs, info = KrylovKit.eigsolve(mat(h_ryd), 10, :SR);

In [ ]:
ΔE = vals[2] - vals[1]

In [ ]:
function get_ΔE(nsites::Int, Δ, Ω, total_time)
    atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)
    h_ryd = rydberg_h(atoms; Δ, Ω) |> attime(total_time)
    
    # Get three smallest eigenvalues
    vals, _, _ = KrylovKit.eigsolve(mat(h_ryd), 3, :SR);
    ΔE = vals[2] - vals[1]
    
    ΔE
end

In [ ]:
get_ΔE(20, Δ, Ω, total_time)

In [ ]:
nsites_list = 3:1:20 |> collect;
ΔEs = [get_ΔE(nsites, Δ, Ω, total_time) for nsites in nsites_list]

In [ ]:
plt.plot(nsites_list, ΔEs)